#Algoritmo AES

---

Escuela Superior de Fisico y Matematicas

Licenciatura En Matematica Algoritmica

**Criptología**

Profesor: Gamaliel Yafte Tellez Sanchez

Alumno: Victor Hugo Martinez Huicochea

#Importamos Librerias

In [ ]:
import numpy as np
import copy

#Funciones para AES

## Gauss-Jordan para $Z_{256}$ (Programa #1 y #2)

In [ ]:
def Gauss(B):
  A=np.array(B,dtype=int)
  AI=np.identity(A.shape[0],dtype=int)
  for i in range(A.shape[0]):
    #Caso 0 en la columna
    if A[i][i]==0:
      #Buscar en las siguientes
      for j in range(i,A.shape[0]):
        #Cuando se halle
        if A[j][i]!=0:
          #cambiar filas en inversa y en original
          A[[i,j]]=A[[j,i]]
          AI[[i,j]]=AI[[j,i]]
          break
        #Caso contrario: No invertible return
        if j==A.shape[0]-1:
          print("No es invertible")
          return

    #Obtenemos la inversa del pivote
    inv=inversa(A[i][i])

    #Multiplicamos para hacer el pivote 1
    A[i]=mulescalar(A[i],inv)
    AI[i]=mulescalar(AI[i],inv)

    #Hacer ceros debajo
    for j in range(i+1,A.shape[0]):
      AI[j]=sumaesc(AI[j],mulescalar(AI[i],A[j][i]))
      A[j]=sumaesc(A[j],mulescalar(A[i],A[j][i]))

    #Hacer ceros Arriba
    for j in range(i-1,-1,-1):
      AI[j]=sumaesc(AI[j],mulescalar(AI[i],A[j][i]))
      A[j]=sumaesc(A[j],mulescalar(A[i],A[j][i]))

  return AI

## Funcion Generadora de LLaves (Programa #3)

In [ ]:
#Genera las 10 llaves del AES
def GenKey(A):
  Aux=copy.deepcopy(A)
  K=[]
  for i in range(10):
    K.append(KeyG(Aux,i))
    Aux=copy.deepcopy(K[i])
  return K

#Genera llaves
def KeyG(A,it):
  #Dividimos en K1 K2 K3 K4
  K=[A[0]>>32,A[0]&4294967295,A[1]>>32,A[1]&4294967295]
  #Trasladamos a K4 K1 K2 K3
  K = K[-1:] + K[:-1]

  #Despues de entrar a S-box
  Kaux=[0,0,0,0]
  #Donde guardaremos la llave
  ki=[0,0]

  #Para cada Ki
  for i in range(len(K)):
    #Dividimos sus 32 bits en 4 Bytes
    for j in range(4):
      kj=(K[i]>>(8*j))&255
      #Cada Byte se manda a la Sbox
      #kj=Sbox(kj)
      kj=LlBox(kj)

      #Se Agrega a la llave
      Kaux[i]=Kaux[i]^(kj<<(8*j))
  #Tenemos T1 T2 T3 T4
  #Procedemos a sacar su Ap para XOR T1
  Ap=algdiv(2**it)
  Kaux[0]=Kaux[0]^Ap

  #Juntamos y enviamos
  ki[0]=(Kaux[0]<<32)^Kaux[1]
  ki[1]=(Kaux[2]<<32)^Kaux[3]
  return ki

def LlBox(a):
  return LBox[a]

## S-Box y su inversa para AES (Programa #4)

In [ ]:
def Sbox(a):
  ai=inversa(a)
  #Convertimos a binario
  ai=np.array(binario(ai)) [np.newaxis]
  #Volvemos fila a Columna
  ai=np.transpose(ai)

  #Multiplicamos por As en Z_2
  ai=As@ai
  #Como es XOR, mod 2 para convertir a binario
  for i in range(len(ai)):
    for j in range(len(ai[0])):
      ai[i][j]=ai[i][j]%2

  #Sumamos el vector Bs en Z_2
  ai=ai+bs
  #como fue XOR, hacemos mod 2 para hacer 0 los pares
  for i in range(len(ai)):
    for j in range(len(ai[0])):
      ai[i][j]=ai[i][j]%2

  #Volvermos de Columna a Fila
  ai=ai.T[0]
  return int(conversor(ai[::-1]))

def SboxINV(a):
  #Convertimos a binario
  ai=binario(a)
  #Volvemos de Fila a Columna
  ai=np.array(ai[::-1]) [np.newaxis]
  ai=np.transpose(ai)

  #Restamos el vector Bs en Z_2 (Como es Z_2 entonces la suma es el inverso tambien)
  ai=ai+bs
  #como fue XOR, hacemos mod 2 para hacer 0 los pares
  for i in range(len(ai)):
    for j in range(len(ai[0])):
      ai[i][j]=ai[i][j]%2

  #Multiplicamos por el inverso de As en Z_2
  ai=Asi@ai
  #Como es XOR, mod 2 para convertir a binario
  for i in range(len(ai)):
    for j in range(len(ai[0])):
      ai[i][j]=ai[i][j]%2

  #De Columnas a Filas
  ai=ai.T[0]
  #Convertimos a decimal
  ai=int(conversor(ai))
  #retornamos el inverso
  return int(inversa(ai))


##Funciones auxiliares

In [ ]:
#Multiplica matrices en Z256
def MultMat(A,B):
  C=np.zeros((len(A),len(B[0])),dtype=int)
  for i in range(len(A)):
    for j in range(len(B[0])):
      for k in range(len(A[0])):
        C[i][j]=suma(C[i][j],multiplicacion(A[i][k],B[k][j]))
  return C

#Multiplicacion en Z256
def multiplicacion(a,b):
  c=0
  for i in range(7,-1,-1):
    if(b&(2**i)):
      c=(a<<i)^c
  if(c>255):
    c=algdiv(c)
  return c

#Multiplicacion por escalar en Z256 (a es un vector y b un escalar)
def mulescalar(a,b):
  c=copy.deepcopy(a)
  for i in range(len(a)):
    c[i]=multiplicacion(a[i],b)
  return c

#Algoritmo de la division para irreducible
def algdiv(g):
  irr=283
  while(g>255):
    i=0
    while(g>=2**i):
      i+=1
    #grado i-1
    g=g^(irr<<((i-1)-8))
  return g

#Encuentra la inversa en Z256
def inversa(a):
  return Inv[a]

#Suma de vectores
def sumaesc(a,b):
  c=copy.deepcopy(a)
  for i in range(len(a)):
    c[i]=suma(a[i],b[i])
  return c

#Suma en Z256
def suma(a,b):
  return a^b

#convierte decimal a binario
def binario(a):
  bin=[]
  if a==0:
    return [0,0,0,0,0,0,0,0]
  while(a!=0):
    bin.append(a%2)
    a=int(a/2)
  #Mientras no sea un byte
  while(len(bin)<8):
    bin.append(0)
  bin=bin[::-1]
  return bin

#Convierte binario a decimal
def conversor(f):
  sum=0
  for i in range(0,len(f)):
    sum+=f[len(f)-i-1]*2**(i)
  return sum

#Guardamos Elementos importantes


## Variables Clave en S-Box

In [ ]:
#Tabla de conversion, tomando en total 256 elementos
Inv=[0, 1, 141, 246, 203, 82, 123, 209, 232, 79, 41, 192, 176, 225, 229, 199, 116, 180, 170, 75, 153, 43, 96, 95, 88, 63, 253, 204, 255, 64, 238, 178, 58, 110, 90, 241, 85, 77, 168, 201, 193, 10, 152, 21, 48, 68, 162, 194, 44, 69, 146, 108, 243, 57, 102, 66, 242, 53, 32, 111, 119, 187, 89, 25, 29, 254, 55, 103, 45, 49, 245, 105, 167, 100, 171, 19, 84, 37, 233, 9, 237, 92, 5, 202, 76, 36, 135, 191, 24, 62, 34, 240, 81, 236, 97, 23, 22, 94, 175, 211, 73, 166, 54, 67, 244, 71, 145, 223, 51, 147, 33, 59, 121, 183, 151, 133, 16, 181, 186, 60, 182, 112, 208, 6, 161, 250, 129, 130, 131, 126, 127, 128, 150, 115, 190, 86, 155, 158, 149, 217, 247, 2, 185, 164, 222, 106, 50, 109, 216, 138, 132, 114, 42, 20, 159, 136, 249, 220, 137, 154, 251, 124, 46, 195, 143, 184, 101, 72, 38, 200, 18, 74, 206, 231, 210, 98, 12, 224, 31, 239, 17, 117, 120, 113, 165, 142, 118, 61, 189, 188, 134, 87, 11, 40, 47, 163, 218, 212, 228, 15, 169, 39, 83, 4, 27, 252, 172, 230, 122, 7, 174, 99, 197, 219, 226, 234, 148, 139, 196, 213, 157, 248, 144, 107, 177, 13, 214, 235, 198, 14, 207, 173, 8, 78, 215, 227, 93, 80, 30, 179, 91, 35, 56, 52, 104, 70, 3, 140, 221, 156, 125, 160, 205, 26, 65, 28]
#Bs del S-box
bs=np.array([[1],
     [1],
     [0],
     [0],
     [1],
     [1],
     [0],
     [0]])
#As del S-box
As=np.array([[1,1,1,0,0,0,1,1],
              [1,1,1,1,0,0,0,1],
              [1,1,1,1,1,0,0,0],
              [0,1,1,1,1,1,0,0],
              [0,0,1,1,1,1,1,0],
              [0,0,0,1,1,1,1,1],
              [1,0,0,0,1,1,1,1],
              [1,1,0,0,0,1,1,1]])
#Inversa de As para inversa de Sbox (Como As son 0 y 1, entonces el G_J servira igual [Inverso de 1 es el mismo y se restan en 0 y 1])
Asi=Gauss(As)

## Variables Clave en AES General

In [ ]:
#Creamos Llaves
LLave=[18446744073709551615,18446744073709551615]
Llave=[9924425342322504201,232422311231232]
Llave=[454761243,454761243]
Llave=[1953184666628070171,1953184666628070171]
#Lbox
LBox=[42, 15, 100, 241, 204, 113, 107, 245, 123, 119, 214, 62, 117, 70, 109, 211, 85, 152, 203, 161, 30, 124, 158, 32, 186, 126, 37, 49, 146, 178, 136, 121, 78, 13, 95, 2, 131, 99, 236, 97, 89, 230, 195, 206, 90, 166, 133, 22, 16, 147, 243, 91, 192, 180, 38, 128, 35, 249, 156, 145, 217, 24, 191, 83, 103, 187, 76, 201, 74, 77, 65, 40, 144, 18, 108, 114, 34, 149, 106, 224, 207, 112, 51, 160, 45, 182, 101, 209, 127, 56, 185, 239, 233, 190, 202, 52, 212, 174, 63, 221, 197, 29, 157, 132, 72, 198, 47, 36, 9, 255, 205, 254, 172, 134, 17, 10, 153, 216, 181, 199, 148, 247, 54, 234, 240, 210, 175, 23, 71, 244, 105, 170, 5, 6, 61, 200, 12, 179, 98, 27, 142, 229, 68, 0, 196, 227, 218, 232, 213, 20, 159, 111, 116, 31, 220, 162, 3, 94, 25, 57, 41, 138, 58, 118, 96, 11, 39, 21, 50, 93, 129, 14, 102, 150, 169, 67, 1, 176, 235, 188, 80, 122, 154, 19, 151, 43, 251, 69, 242, 208, 82, 194, 223, 143, 173, 110, 246, 86, 171, 46, 177, 237, 48, 141, 4, 250, 115, 248, 253, 183, 167, 189, 33, 84, 219, 26, 238, 60, 222, 225, 55, 81, 120, 184, 165, 8, 44, 164, 168, 125, 73, 87, 137, 66, 139, 7, 92, 135, 64, 228, 104, 252, 75, 226, 155, 28, 59, 130, 163, 193, 53, 140, 231, 88, 79, 215]

In [ ]:
#Al que se le encontrara su inversa
A=[[1,4,3,3],
   [2,0,1,3],
   [2,3,2,1],
   [2,3,5,6]]
A=[[2,7,3,13],
   [13,7,1,9],
   [15,6,5,11],
   [13,3,3,13]]

AI=Gauss(A)
##Comprobacion de Que AI es inversa de A
print("Matriz A:\n ",np.array(A),"\n Matriz AI:\n",AI)
print("Multiplicado da:\n",MultMat(A,AI))

Matriz A:
  [[ 2  7  3 13]
 [13  7  1  9]
 [15  6  5 11]
 [13  3  3 13]] 
 Matriz AI:
 [[245 119 185   8]
 [206 116 228 213]
 [209  97  19 196]
 [ 43   2  96 124]]
Multiplicado da:
 [[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]


# AES (Programa #5)

In [ ]:
#Ejemplos usados
#M=[18446744073709551615,18446744073709551615]
#M=[1953184666628070171,1953184666628070171]
#M=[18376798761751380251,18376798761751380251]
#M=[133232423432424234,213123123123123213]
M=[0,1]
#M=[255,256]
print("Mensaje Original:",M)

#Encriptado
#Generamos las llaves
GK=GenKey(Llave)
for it in range(10):
  M4=[]
  #Convertimos a M4
  for i in range(2):
    Maux=[]
    for j in range(4):
      Maux.append((M[0]>>(8*(4*i+j)))&255)
    M4.append(Maux)
  for i in range(2):
    Maux=[]
    for j in range(4):
      Maux.append((M[1]>>(8*(4*i+j)))&255)
    M4.append(Maux)
  M4=np.array(M4,dtype=int)

  #Pasamos por sbox
  for i in range(4):
    for j in range(4):
      M4[i][j]=Sbox(M4[i][j])


  #Mexcla de filas
  M4[1]=np.roll(M4[1],3)
  M4[2]=np.roll(M4[2],2)
  M4[3]=np.roll(M4[3],1)

  #Mezcla de Columnas
  M4=MultMat(A,M4)

  #Reinvertimos mensaje:
  C=[0,0]
  for i in range(2):
    for j in range(4):
      C[0]=C[0]^(int(M4[i][j])<<(8*(4*i+j)))
  for i in range(2):
    for j in range(4):
      C[1]=C[1]^(int(M4[i+2][j])<<(8*(4*i+j)))

  #Hacemos XOR con la llave de esta iteracion
  ki=GK[it]
  C[0]=C[0]^ki[0]
  C[1]=C[1]^ki[1]
  #Copiamos para siguiente ireracion
  M=copy.deepcopy(C)
  #print("Cifrado de iteracion: ", it, "Dando resultado:",C)

print("Mensaje Cifrado:",C)

Mensaje Original: [0, 1]
Mensaje Cifrado: [996156352679900960, 11728750728847474366]


#Desencriptado del AES (Programa #6)

In [ ]:
print("Mensaje Cifrado:",C)

#Desencriptado
#Generamos llaves
GK=GenKey(Llave)
for it in range(10):
  #Hacemos XOR con la llave de la iteracion correspondiente
  ki=GK[9-it]
  C[0]=C[0]^ki[0]
  C[1]=C[1]^ki[1]

  M4=[]
  #Convertimos a M4
  for i in range(2):
    Maux=[]
    for j in range(4):
      Maux.append((C[0]>>(8*(4*i+j)))&255)
    M4.append(Maux)
  for i in range(2):
    Maux=[]
    for j in range(4):
      Maux.append((C[1]>>(8*(4*i+j)))&255)
    M4.append(Maux)
  M4=np.array(M4,dtype=int)

  #Multiplicamos por inversa de A
  M4=MultMat(AI,M4)

  #Regresamos de mezcla de filas
  M4[1]=np.roll(M4[1],-3)
  M4[2]=np.roll(M4[2],-2)
  M4[3]=np.roll(M4[3],-1)

  #Pasamos por la inversa de S-Box
  for i in range(4):
    for j in range(4):
      M4[i][j]=SboxINV(M4[i][j])

  #Reinvertimos mensaje:
  M=[0,0]
  for i in range(2):
    for j in range(4):
      M[0]=M[0]^(int(M4[i][j])<<(8*(4*i+j)))
  for i in range(2):
    for j in range(4):
      M[1]=M[1]^(int(M4[i+2][j])<<(8*(4*i+j)))

  #Copiamos para siguiente Iteracion
  C=copy.deepcopy(M)
  #print("Cifrado de iteracion: ", it, "Dando resultado:",C)
print("mensaje Descifrado:",M)

Mensaje Cifrado: [996156352679900960, 11728750728847474366]
mensaje Descifrado: [0, 1]
